<a href="https://colab.research.google.com/github/1konlawat/1konlawat/blob/main/Model_bonus_employee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
import pandas as pd

# 1. โหลดข้อมูล (สมมติว่ายังมีตัวแปร df อยู่)
# df = pd.read_csv('employee_ml_1M_rows.csv')

# --- ส่วนที่ 1: เติมข้อมูลประเภท "ข้อความ" (Categorical) ---
# เราจะเติมด้วยคำว่า 'Unknown' เพื่อให้ AI รู้ว่าตรงนี้ไม่มีข้อมูลนะ แต่ยังเก็บแถวนี้ไว้
cat_cols = ['department', 'position_level', 'province', 'company_size', 'employment_type']
for col in cat_cols:
    df[col] = df[col].fillna('Unknown')

# --- ส่วนที่ 2: เติมข้อมูลประเภท "ตัวเลข" (Numerical) ---
# อายุ (Age) เติมด้วยค่ามัธยฐาน (ค่ากลาง)
df['age'] = df['age'].fillna(df['age'].median())

# คะแนนประเมิน (Performance Score) เติมด้วยคะแนนกลางๆ เช่น 3.0
df['performance_score'] = df['performance_score'].fillna(3.0)

# โบนัส (Bonus) ถ้าไม่มีข้อมูล ให้ตีเป็น 0 ไปเลย
df['bonus'] = df['bonus'].fillna(0)

# --- ส่วนที่ 3: จัดการ ID และ ชื่อ (ถ้ายังอยากเก็บไว้) ---
df['employee_id'] = df['employee_id'].fillna('EMP_UNDEFINED')
df['employee_name'] = df['employee_name'].fillna('Anonymous')

# เช็คผลลัพธ์
print(f"จำนวนแถวคงเหลือ: {len(df):,}") # จะต้องได้ 1,000,000 แถวเท่าเดิมเป๊ะ!
print(f"ค่าว่างคงเหลือในระบบ: {df.isnull().sum().sum()}")


In [ ]:
%whos

Interactive namespace is empty.


In [ ]:
# เปลี่ยนชื่อไฟล์ตามที่คุณต้องการ
df.to_csv('employee_ml_1M_rows_cleaned.csv', index=False)

In [ ]:
from google.colab import files
files.download('employee_ml_1M_rows_cleaned.csv')


In [ ]:
from google.colab import files
upload = files.upload()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. โหลดข้อมูล
df = pd.read_csv('employee_ml_1M_rows_cleaned.csv')

# 2. การเตรียมข้อมูล (Preprocessing)
# เลือกคอลัมน์ที่จะใช้เป็นฟีเจอร์ (X) และเป้าหมาย (y)
# เราจะทำนาย 'bonus' โดยใช้ปัจจัยต่างๆ
features = ['experience', 'age', 'department', 'position_level',
            'province', 'company_size', 'performance_score', 'employment_type']
X = df[features].copy()
y = df['bonus'].astype(str)  # แปลงเป็น string เพื่อทำ Classification (เนื่องจากมีแค่ 2 ค่าหลัก)

# แปลงข้อมูลตัวอักษรให้เป็นตัวเลข (Label Encoding)
le_dict = {}
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    le_dict[col] = le
    print(f"Encoded {col}")

# 3. แบ่งข้อมูลเป็นชุด Train และ Test
# หมายเหตุ: ข้อมูลมี 1 ล้านแถว หากรันในเครื่องทั่วไปอาจใช้เวลานาน
# ในตัวอย่างนี้ผมจะสุ่มมา 200,000 แถวเพื่อให้ประมวลผลไวขึ้น
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=200000, stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

# 4. สร้างและฝึกสอนโมเดล (Train Model)
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# 5. ประเมินผลโมเดล
y_pred = model.predict(X_test)
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 6. ดูความสำคัญของปัจจัยต่างๆ (Feature Importance)
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=importances.values, y=importances.index)
plt.title('ปัจจัยที่มีผลต่อการทำนายโบนัส')
plt.show()

# --- ตัวอย่างการนำไปใช้ทำนายข้อมูลใหม่ ---
# สมมติข้อมูลพนักงานใหม่
new_employee = pd.DataFrame([{
    'experience': 5.0,
    'age': 30,
    'department': 'IT',
    'position_level': 'Manager',
    'province': 'กรุงเทพ',
    'company_size': 'Large',
    'performance_score': 5.0,
    'employment_type': 'Full-time'
}])

# ต้องแปลงข้อมูลพนักงานใหม่ให้เป็นตัวเลขเหมือนตอน Train
for col, le in le_dict.items():
    new_employee[col] = le.transform(new_employee[col].astype(str))

prediction = model.predict(new_employee)
print(f"\nผลการทำนายโบนัสสำหรับพนักงานใหม่: {prediction[0]}")
